@######################error rienvforenfv

In [1]:
import os
import pandas as pd
import pickle
import re
import shutil
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from spellchecker import SpellChecker
import nbformat
from nbconvert import PythonExporter
from flask import Flask, render_template, request
import sys
import subprocess
from googlesearch import search


app = Flask(__name__, template_folder="templates", static_folder="static")
app.config["UPLOAD_FOLDER"]=r"static"

# Function to check and install missing libraries
def install_missing_libraries():
    required_libraries = [
        "wikipedia",
        "textblob",
        "spellchecker",
        "requests",
        "bs4",
        "termcolor",
        "Pillow",
        "matplotlib",
        "numpy",
        "wikipedia",
        "google"
    ]
    missing_libraries = []
    for lib in required_libraries:
        try:
            __import__(lib)
        except ImportError:
            missing_libraries.append(lib)
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
    if missing_libraries:
        print("Installed missing libraries:", missing_libraries)
    else:
        print("All required libraries are installed.")

# Convert Jupyter Notebook cell to Python script
def notebook_to_python(notebook_cell):
    notebook_content = nbformat.v4.new_notebook(cells=[nbformat.v4.new_code_cell(notebook_cell)])
    python_exporter = PythonExporter()
    python_script, _ = python_exporter.from_notebook_node(notebook_content)
    # Fix indentation and escape newline characters
    python_script = python_script.replace('\n', '\\n')
    return python_script

# Preprocessing function
def preprocess_input(user_input):
    user_input = user_input.lower()  # Convert to lowercase
    spell = SpellChecker()
    tokens = TextBlob(user_input).words  # Tokenization
    corrected_tokens = []
    for token in tokens:
        # Skip spelling correction for non-English words
        if re.match(r'^[a-zA-Z]+$', token):
            corrected_tokens.append(spell.correction(token))
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)

# Define search_book function
def search_book(book_title, user_query):
    import wikipedia
    try:
        page = wikipedia.page(book_title)
        content = page.content
        relevant_content = extract_relevant_content(content, user_query)
        
        # Extract multiple image URLs from Google Images
        image_urls = fetch_image_urls(book_title, 4)  # Fetch four images
        if not image_urls:
            return "Sorry, couldn't find any images for this book.", None, None
        
        # Fetch top 3 Google search links
        search_links = fetch_google_search_links(user_query, 3)
        
        import os

        # Download the images and save them locally
        image_paths = []
        for i, url in enumerate(image_urls):
            image_path = f"{book_title.replace(' ', '_')}_image_{i}.png"
            response = requests.get(url, stream=True)
            full_image_path = os.path.join(app.config["UPLOAD_FOLDER"], image_path)
            with open(full_image_path, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            image_paths.append(image_path)
        
        return relevant_content, image_paths, search_links
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Multiple results found. Please be more specific: {', '.join(e.options)}", None, None
    except wikipedia.exceptions.PageError:
        return "Sorry, I couldn't find any information about that book.", None, None

def extract_relevant_content(content, user_query):
    relevant_content = ""
    sections = re.split(r'\n(?===)', content)  # Split content into sections
    for section in sections:
        if user_query.lower() in section.lower():
            relevant_content += section + '\n'
    return relevant_content

def fetch_image_urls(query, num_images):
    try:
        search_url = f"https://www.google.com/search?q={query.replace(' ', '+')}+book+cover&tbm=isch"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        img_tags = soup.find_all('img')
        # Extract URLs of the first 'num_images' images
        image_urls = [img['src'] for img in img_tags[1:num_images+1]]
        return image_urls
    except Exception as e:
        print("Error fetching image URLs:", e)
        return []

def fetch_google_search_links(query, num_links):
    try:
        search_links = list(search(query, num_results=num_links, stop=num_links, pause=2))
        return search_links
    except Exception as e:
        print("Error fetching Google search links:", e)
        return []

# Notebook cell content converted to Python script
notebook_cell = """
def preprocess_input(user_input):
    user_input = user_input.lower()  # Convert to lowercase
    spell = SpellChecker()
    tokens = TextBlob(user_input).words  # Tokenization
    corrected_tokens = []
    for token in tokens:
        # Skip spelling correction for non-English words
        if re.match(r'^[a-zA-Z]+$', token):
            corrected_tokens.append(spell.correction(token))
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)
"""



# Define the function to load pickled objects
def load_pickled_objects():
    with open(r'E:\semsester 6\WEB_PROGRAMMING_PROJECT\recommend_book_function.pkl', 'rb') as f:
        recommend_book = pickle.load(f)

    with open(r'E:\semsester 6\WEB_PROGRAMMING_PROJECT\knn_model.pkl', 'rb') as f:
        knn_model = pickle.load(f)
    
    return recommend_book, knn_model


# Define route for book recommendations
@app.route('/Recommendation.html', methods=['GET', 'POST'])
def recommendation():
    if request.method == 'POST':
        book_name = request.form['book_name']
        recommendations = recommend_book(book_name)
        return render_template('Recommendation.html', recommendations=recommendations)
    return render_template('Recommendation.html', recommendations=None)


def recommend_book(book_name):
    # Load necessary data and models
    corrected_books_df = pd.read_csv(r'C:\Users\Dell\Downloads\BX-Books-Corrected.csv', sep=';', encoding='latin-1')

    # Check if 'title' column exists
    if 'title' not in corrected_books_df.columns:
        return "Error: 'title' column not found in the DataFrame."

    # Assuming knn_model is already loaded
    matched_book_name = book_name.strip().lower()
    if matched_book_name not in corrected_books_df['title'].str.lower().values:
        return []  # Return an empty list if book not found

    # Find recommendations based on similarity using knn_model
    query_book = corrected_books_df[corrected_books_df['title'].str.lower() == matched_book_name]
    query_features = query_book[['year', 'publisher']]  # Assuming 'year' and 'publisher' are used for similarity

    distances, indices = knn_model.kneighbors(query_features, n_neighbors=6)

    recommendations = []
    for i in range(1, len(distances.flatten())):
        recommended_book_index = indices.flatten()[i]
        recommended_book = corrected_books_df.iloc[recommended_book_index]
        recommendations.append(recommended_book)

    return recommendations



@app.route('/')
def home():
    return render_template('HOME.html')


@app.route('/HOME.html')
def home2():
    return render_template('HOME.html')
@app.route('/Self_help')
def self_help():
    return render_template('Self_help.html')

@app.route('/Romance.html')
def romance():
    return render_template('Romance.html')

@app.route('/Fantasy.html')
def fantasy():
    return render_template('Fantasy.html')

@app.route('/Comics.html')
def comics():
    return render_template('Comics.html')

@app.route('/add_to_cart.html')
def add_to_cart():
    return render_template('add_to_cart.html')

@app.route('/Search.html', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        user_input = request.form['user_input']
        # Convert Jupyter Notebook cell to Python script
        python_script = notebook_to_python(notebook_cell)
        # Execute the Python script in the same environment
        exec(python_script)
        preprocessed_input = preprocess_input(user_input)
        response, images, search_links = search_book(preprocessed_input, user_input)
        print(images)
        return render_template('Search.html', response=response, images=images, search_links=search_links)
    return render_template('Search.html')


# Main block to run the app
if __name__ == '__main__':
    recommend_book, knn_model = load_pickled_objects()
    app.run(debug=True, use_reloader=False)


c:\Users\Dell\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Dell\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2024 22:54:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:54:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Jun/2024 22:54:58] "GET /Comics.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:08] "GET /Recommendation.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:09] "GET /Search.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:11] "GET /Romance.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:36] "GET /HOME.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:53] "GET /Fantasy.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:55:56] "GET /Self_help HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:56:19] "GET /Romance.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:56:21] "GET /Fantasy.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:56:29] "GET /add_to_cart.html HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:57:04] "GET /order_data.json HTTP/1.1" 404 -
127.0.0.1 - - 

Error fetching Google search links: search() got an unexpected keyword argument 'num_results'


127.0.0.1 - - [19/Jun/2024 22:57:36] "POST /Search.html HTTP/1.1" 200 -


['solo_leveling_image_0.png', 'solo_leveling_image_1.png', 'solo_leveling_image_2.png', 'solo_leveling_image_3.png']


127.0.0.1 - - [19/Jun/2024 23:02:09] "GET /HOME.html HTTP/1.1" 200 -
